# Metody Obliczeniowe w Nauce i Technice Laboratorium 10
## DFT, IDFT
### Paweł Gorgolewski

In [1]:
import numpy as np

1.1 FFT

In [2]:
def create_F_matrix(n):
    F = np.zeros((n,n), dtype=complex)
    for i in range(n):
        for j in range(n):
            # see http://webhome.phy.duke.edu/~rgb/Class/phy51/phy51/node15.html
            F[i][j] = np.exp(-1j * 2 * np.pi * i * j / n)

    return F

def FFT(x: np.array):
    n = len(x)
    return create_F_matrix(n) @ x

sprawdzenie poprawnosci

In [3]:
x = list(range(100))

my_fft = FFT(x)
np_fft = np.fft.fft(x)

np.testing.assert_allclose(my_fft, np_fft)

Algorytm jest poprawny, assert nie zgłasza błędu

1.2 IDFT

In [4]:
#https://numpy.org/doc/stable/reference/generated/numpy.conjugate.html#:~:text=2)%0A%3E%3E%3E-,np.conjugate(x),-array(%5B%5B%201.%2D1
def IDFT(y: np.array):
    F = create_F_matrix(len(y))
    F_inv = np.conj(F) / len(y)
    return F_inv @ y

sprawdzanie poprawności

In [5]:
my_idft = IDFT(np_fft)
np_idft = np.fft.ifft(np_fft)

np.testing.assert_allclose(my_idft, np_idft, atol=1e-05)

1.3 algorytm Cooleya-Turkeya -> można zoptymalizować

In [6]:
def cooley_turkey_fft(x: np.array):
    def create_S_diag(n: int):
        S_diag = np.zeros(n, dtype=complex)
        for i in range(n):
            S_diag[i] = np.exp(-1j * np.pi * i / n)

        return S_diag

    n = len(x)
    if n == 2:
        return create_F_matrix(2) @ x

    even, odd = cooley_turkey_fft(x[0::2]), cooley_turkey_fft(x[1::2])
    S_diag = create_S_diag(n // 2)
    return np.array([even[k] + S_diag[k] * odd[k] for k in range(n // 2)] +
                    [even[k] - S_diag[k] * odd[k] for k in range(n // 2)])

In [7]:
x = np.array([1,2,3,4,5,6,7,8])

my_fft = cooley_turkey_fft(x)
np_fft = np.fft.fft(x)

np.testing.assert_allclose(my_fft, np_fft)